In [1]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Tuple
from rich import print
from rich.progress import track
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score,
    average_precision_score,
    precision_recall_curve,
)
from sklearn.inspection import permutation_importance
from scipy.interpolate import UnivariateSpline
from plotly import express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
import skops.io as skio

# Data preparation

In [2]:
# Load the data
df = pd.read_csv("./data.set")
df.head()

,Id,AbdDiameter,EsdDiameter,AspectRatio,AbdArea,AbdVolume,Transparency,Elongation,Compactness,Ch1Peak,Ch2Peak,ConvexPerimeter,Perimeter,Intensity,SigmaIntensity,SumIntensity,Roughness,TAXA,DATE,class
0,36,28.60,15.10,0.12,179.14,12200.0,0.47,11.09,4.20,2688,91,89.89,97.19,44.67,10.84,19834,1.08,diat,20071115,False
1,37,16.77,10.26,0.18,82.71,2470.0,0.39,9.74,3.77,2688,91,52.71,62.58,38.75,7.40,7943,1.19,diat,20071115,False
2,38,19.28,11.53,0.16,104.50,3750.0,0.40,8.96,3.53,2688,91,60.62,68.04,43.51,8.94,11269,1.12,diat,20071115,False
3,39,18.01,10.82,0.17,91.99,3060.0,0.40,8.50,3.38,2688,91,56.62,62.49,42.21,7.38,9624,1.10,diat,20071115,False
4,133,89.15,44.93,0.17,1585.67,371000.0,0.50,114.25,37.01,3083,82,280.25,858.70,59.09,19.38,232225,3.06,diat,20071115,False


In [3]:
# Get training features and labels
# Here I ignore size-related variables, e.g., diameter, area, volume, etc.
X = df[
    [
        "AspectRatio",
        "Transparency",
        "Elongation",
        "Compactness",
        "Ch1Peak",
        "Ch2Peak",
        "Intensity",
        "SigmaIntensity",
        "Roughness",
    ]
]
y = df["class"]

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_true = train_test_split(
    X, y, test_size=0.3, random_state=42
)  # the random state is for reproducibility

# Model initialization and training

In [5]:
# Create a pipeline
pipeline = make_pipeline(StandardScaler(), LogisticRegression())

In [6]:
# Fit the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

# Model evaluation

In [7]:
# Predict the labels
y_pred = pipeline.predict(X_test)

In [8]:
# Get the probabilities for the positive class
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model accuracy
print(
    f"{'accuracy:':<20}{pipeline.score(X_test, y_true):.2f}\n"
    f"{'precision:':<20}{precision_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'recall:':<20}{recall_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'f1:':<20}{f1_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'average precision:':<20}{average_precision_score(y_true, y_proba):.2f}"
)

accuracy:           0.85
precision:          0.87
recall:             0.93
f1:                 0.90
average precision:  0.93

In [9]:
# print classification report
print(
    classification_report(
        y_true=y_true,
        y_pred=y_pred,
        target_names=["object", "detritus"],
        zero_division=0,
    )
)

precision    recall  f1-score   support

      object       0.78      0.64      0.70       676
    detritus       0.87      0.93      0.90      1766

    accuracy                           0.85      2442
   macro avg       0.82      0.78      0.80      2442
weighted avg       0.85      0.85      0.84      2442

# Save Model

In [10]:
# set the model path
model_dir = Path("./model")

# create the model directory if it does not exist
model_dir.mkdir(exist_ok=True)

In [11]:
# Save the model
model_file = "model.skops"
skio.dump(pipeline, model_dir.joinpath(model_file))

In [12]:
# Save model configuration
# ModelConfig class
class ModelConfig:
    def __init__(
        self,
        name: str,
        version: str,
        framework: str,
        class_names: list[str],
        input_shape: Tuple[int, int, int],
        features: list[str],
    ) -> None:
        self.Name: str = name
        self.Version: str = version
        self.Framework: str = framework
        self.Class_names: list[str] = class_names
        self.Input_shape: list[int] = list(input_shape)
        self.Features: list[str] = features

        # Check if any value is None
        if any(value is None for value in self.__dict__.values()):
            raise ValueError("Not all values have been initialized")

    # representation of the class
    def __repr__(self) -> str:
        return (
            "ModelConfig(\n"
            f"{'Name=':<20}{self.Name},\n"
            f"{'Version=':<20}{self.Version},\n"
            f"{'Framework=':<20}{self.Framework},\n"
            f"{'Class_names=':<20}{self.Class_names},\n"
            f"{'Input_shape=':<20}{self.Input_shape},\n"
            f"{'Features=':<20}{self.Features}\n"
            ")"
        )

In [13]:
# create the model configurations
model_configurations = ModelConfig(
    name="detritus_classifier",
    version="1",
    framework="skops",
    class_names=["object", "detritus"],
    input_shape=[],
    features=pipeline.feature_names_in_.tolist(),
)
print(model_configurations)

ModelConfig(
Name=               detritus_classifier,
Version=            1,
Framework=          skops,
Class_names=        ['object', 'detritus'],
Input_shape=        [],
Features=           ['AspectRatio', 'Transparency', 'Elongation', 'Compactness', 'Ch1Peak', 'Ch2Peak', 'Intensity',
'SigmaIntensity', 'Roughness']
)

In [14]:
# save model configuration to json
model_config_file = model_dir.joinpath("model_config.json")
with open(model_config_file, "w") as f:
    json.dump(model_configurations.__dict__, f, indent=4)

In [ ]:
# Load the model and test
pipeline = skio.load(model_dir.joinpath(model_file))

# Predict the labels
y_pred = pipeline.predict(X_test)

# Get the probabilities for the positive class
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model accuracy
print(
    f"{'accuracy:':<20}{pipeline.score(X_test, y_true):.2f}\n"
    f"{'precision:':<20}{precision_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'recall:':<20}{recall_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'f1:':<20}{f1_score(y_true, y_pred, average='binary'):.2f}\n"
    f"{'average precision:':<20}{average_precision_score(y_true, y_proba):.2f}"
)

accuracy: 0.85

precision: 0.87

recall: 0.93

f1: 0.90

average precision: 0.93

In [ ]:
def plot_permutation_importances(pipeline, X_test, y_true):
    result = permutation_importance(
        pipeline, X_test, y_true, n_repeats=30, random_state=0
    )
    sorted_importances_idx = result.importances_mean.argsort()
    importances = pd.DataFrame(
        result.importances[sorted_importances_idx].T,
        columns=X_test.columns[sorted_importances_idx],
    )
    ax = importances.plot.box(vert=False, whis=10)
    ax.set_title("Permutation Importances (test set)")
    ax.axvline(x=0, color="k", linestyle="--")
    ax.set_xlabel("Decrease in accuracy score")
    ax.figure.tight_layout()

In [ ]:
plot_permutation_importances(pipeline, X_test, y_true)

In [ ]:
# plot confusion matrix
def plot_confusion_matrix(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    class_names: list[str],
    normalize: bool = True,
):
    cm = confusion_matrix(
        y_true=y_true,
        y_pred=y_pred,
        normalize="true" if normalize else None,
    )

    fig = go.Figure(
        data=go.Heatmap(
            z=cm,
            x=class_names,
            y=class_names,
            colorscale="Blues",
            showscale=False,
            text=cm,
            texttemplate="%{text:.2f}",
            textfont={"size": 20},
        )
    )

    fig.update_layout(
        title="Normalized Confusion Matrix" if normalize else "Confusion Matrix",
        xaxis_title="Predicted",
        yaxis_title="True",
        autosize=True,
    )

    fig.show()

In [ ]:
plot_confusion_matrix(y_true=y_true, y_pred=y_pred, class_names=["object", "detritus"])

In [ ]:
# plot precision-recall curve
def plot_precision_recall_curve(y_true: np.ndarray, y_proba: np.ndarray):
    precision, recall, _ = precision_recall_curve(y_true, y_proba)

    fig = px.line(
        x=recall,
        y=precision,
        labels={"x": "Recall", "y": "Precision"},
        title="Precision-Recall Curve",
    )

    fig.update_traces(mode="lines+markers")
    fig.show()

In [ ]:
plot_precision_recall_curve(y_true=y_true, y_proba=y_proba)

# Train test size effect

In [ ]:
# plot metrics as function of training set size
def plot_metrics(train_sizes, metric_list, _label, _color):
    plt.plot(
        train_sizes,
        metric_list,
        color=_color,
        marker=".",
        linestyle="",
        alpha=0.2,
        label=None,
        markersize=2,
    )

    # Smoothing the accuracy with a spline
    spline = UnivariateSpline(
        train_sizes, metric_list, s=0.5
    )  # s is the smoothing factor

    # Generate x values for the spline line
    x_smooth = np.linspace(train_sizes.min(), train_sizes.max(), 500)
    y_smooth = spline(x_smooth)

    # Calculate the confidence interval (using standard error of the mean)
    confidence_interval = 1.96 * np.std(metric_list) / np.sqrt(len(metric_list))

    # Calculate the upper and lower bounds for the confidence interval
    upper_bound = y_smooth + confidence_interval
    lower_bound = y_smooth - confidence_interval

    # Plotting the smooth spline
    plt.plot(x_smooth, y_smooth, label=_label, color=_color, linestyle="-")

    # Adding the confidence interval
    plt.fill_between(
        x_smooth, upper_bound, lower_bound, color=_color, alpha=0.2, label=None
    )

In [ ]:
# Effect of training data set
train_sizes = np.sort(np.random.uniform(0.01, 0.3, 1000))  # 1000 random samples sorted
accuracy_list = []
precision_list = []
recall_list = []
avg_precision_list = []

# Iterate over different training sizes
for train_size in track(train_sizes):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size
    )  # , random_state=42)

    # Fit the pipeline on the training data
    pipeline.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = pipeline.predict(X_test)

    # Calculate metrics
    accuracy = pipeline.score(X_test, y_test)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    avg_precision = average_precision_score(y_test, y_pred)

    # Append metrics to the lists
    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    avg_precision_list.append(avg_precision)

# Create a single plot for all metrics
plt.figure(figsize=(6, 4))
plot_metrics(train_sizes, accuracy_list, "Accuracy", "blue")
plot_metrics(train_sizes, precision_list, "Precision", "orange")
plot_metrics(train_sizes, recall_list, "Recall", "green")
plot_metrics(train_sizes, avg_precision_list, "Average Precision", "red")

# Adding titles and labels
plt.title("Metrics vs. Training Size")
plt.xlabel("Training Size")
plt.ylabel("Score")
# plt.xscale("log")
# plt.ylim(0, 1)

# Show legend
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()